In [6]:
import tensorflow as tf
from tensorflow import keras
import cv2
import pandas as pd
import mediapipe as mp
import numpy as np
import os

In [17]:
csv_path=os.path.join(os.getcwd(),os.path.join('Datasets','custom_dataset.csv'))
df=pd.read_csv(csv_path)

In [18]:
words=pd.unique(df['value'])
word_dict={index: value for index, value in enumerate(words)}


{0: 'A'}

In [19]:
model_path=os.path.join(os.getcwd(),os.path.join('saved_models','custom_model.h5'))
model=keras.models.load_model(model_path)
print('Loaded custome Neural Net')
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_10 (Dropout)         (None, 42)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 40)                1720      
_________________________________________________________________
dropout_11 (Dropout)         (None, 40)                0         
_________________________________________________________________
dense_16 (Dense)             (None, 30)                1230      
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 31        
Total params: 2,981
Trainable params: 2,981
Non-trainable params: 0
_________________________________________________________________


In [20]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
cap = cv2.VideoCapture(0,cv2.CAP_DSHOW)
rows=[]
with mp_hands.Hands(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5,
    max_num_hands=1) as hands:
    while True:
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
              # If loading a video, use 'break' instead of 'continue'.
            continue
        image=cv2.flip(image,1)
        cv2.putText(image, "Testing ", (80, 400), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
                
            # Flip the image horizontally for a later selfie-view display, and convert
            # the BGR image to RGB.
        
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                # To improve performance, optionally mark the image as not writeable to
                # pass by reference.

        image.flags.writeable = False
        results = hands.process(image)

                # Draw the hand annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.multi_hand_landmarks:
            row=[]
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                for hand_landmark in hand_landmarks.landmark:
                    row.append(hand_landmark.x)
                    row.append(hand_landmark.y)
                            
                
                if len(row)<42:
                    break
                row=np.array([row])
                prediction=np.argmax(model.predict(row))
                if prediction not in word_dict.keys():
                    break
                cv2.putText(image,word_dict[prediction],(400,80),cv2.FONT_HERSHEY_SIMPLEX,0.9,(0,255,0),2)
        cv2.imshow('MediaPipe Hands', image)

        if cv2.waitKey(5) & 0xFF == 27:
            break
    cv2.destroyAllWindows()
    cap.release()